In [1]:
import numpy as np
import pickle
from pandas import DataFrame, read_csv
from sklearn import preprocessing
import helperModule as hm
import warnings
import tensorflow as tf

warnings.filterwarnings("ignore")

In [16]:
#########################
############ Hyperparamter
#########################

num_nodes= 512
batch_size = 512
num_features = 41
num_labels = 5
STDDEV = 0.2
learning_rate = 0.01


In [3]:
#########################
############ Read in data
#########################

data_train, data_test = hm.readData()
x_train, y_train, x_test, y_test = hm.splitData(data_train, data_test)


In [4]:
#########################
# Mapping of attack types
#########################

'''
38 different attack types are mapped to their corresponding group
-----------
normal                      ==>   NORMAL
back,land ..                ==>   DOS
satan,ipsweep,nmap, ..      ==>   PROBE
ftp_write,guess_passwd, ..  ==>   R2L
rootkit,perl ..             ==>   U2R
-----------
'''

y_train, y_test = hm.mapAttackTypes(y_train, y_test)


In [5]:
##########################
# Feature Encoding
##########################

'''
Categorial features (protocol_types, service, flags) are encoded into integers.
-----------
protocol_types:
['tcp' 'udp' 'icmp']                          ==> [0, 1, 2]
service:
['ftp_data', 'telnet', ... 'rje', 'harvest']  ==> [0, 1, .... 67, 68]
flags:
['SF', 'S0', ...  ,'S2', 'OTH']               ==> [ 0, 1 ... , 9, 10]
-----------
'''

x_train = hm.encodeFeatures(x_train)
x_test = hm.encodeFeatures(x_test)


In [6]:
##########################
# Label Encoding
##########################

'''
5 different attack groups are encoded into integers
-----------
NORMAL         ==>   0
DOS            ==>   1
PROBE          ==>   2
R2L            ==>   3
U2R            ==>   4
-----------
'''
y_train = hm.encodeLabels(y_train)
y_test = hm.encodeLabels(y_test)

In [7]:
#########################
####### Binarize Labels
#########################

'''
attack groups are binarized into vectors
-----------
NORMAL         ==>   [1,0,0,0,0]
DOS            ==>   [0,1,0,0,0]
..
..
'''

y_train = hm.binarizeLabels(y_train)
y_test = hm.binarizeLabels(y_test)

In [8]:
#########################
######### Feature Scaling
#########################

'''
Scale non-categorial features into to values between 0 and 1
'''

x_train, x_test = hm.scaleFeatures(x_train, x_test) 

#########################
######### Turn data Frame into matrix 
#########################

x_train = x_train.astype(np.float32).values
x_test = x_test.astype(np.float32).values


In [9]:
def give_accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [12]:
def fully_conn(x_tensor, num_outputs):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    input_size = int(x_tensor.shape[1])
    
    #print(input_size)
    
    W = tf.Variable(tf.truncated_normal([input_size, num_outputs], stddev=STDDEV))
    b = tf.Variable(tf.zeros(num_outputs))
    
    x_tensor = tf.add(tf.matmul(x_tensor, W), b)
    x_tensor = tf.nn.relu(x_tensor)
        
        
    return x_tensor, W, b



def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """    
    input_size = int(x_tensor.shape[1])
    
    W = tf.Variable(tf.truncated_normal([input_size, num_outputs], stddev=STDDEV))
    b = tf.Variable(tf.zeros(num_outputs))
    
    x_tensor = tf.add(tf.matmul(x_tensor, W), b)
        
    return x_tensor, W, b



def neural_net(x):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """    
    x, W1, b1 = fully_conn(x, 128)
    x, W2, b2 = fully_conn(x, 128)

    x, W3, b3 = output(x, 5)
    
    weights = [W1, W2, W3]
    bias = [b1, b2, b3] 
        
    return x, weights, bias


def print_stats(session, step, feature_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    
    lost = session.run(loss, feed_dict={
                tf_x_train: feature_batch,
                tf_y_train: label_batch})
    
    #weights = session.run(weights)
    
    
    print('Loss at step %d is: %f' % (step,lost))
    
    validation_accuracy = session.run(accuracy, feed_dict={
                tf_x_train: feature_batch,
                tf_y_train: label_batch})
    
    print('Batch accuracy at step %d is: %f' % (step,validation_accuracy))
    
    pass



In [17]:
graph = tf.Graph()
with graph.as_default():

    
# Inputs  
  tf_x_train = tf.placeholder(tf.float32,shape=(batch_size, num_features))
  tf_y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_x_test = tf.constant(x_test)
  tf_y_test = tf.constant(y_test)  
    
  
    
  # Model
  logits, weights, bias = neural_net(tf_x_train)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_y_train))

  # Optimizer.
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)
  

  train_prediction = tf.nn.softmax(logits)
  correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(tf_y_train, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

  W1 = weights[0]
  W2 = weights[1]
  W3 = weights[2]
  b1 = bias[0]
  b2 = bias[1]
  b3 = bias[2]

  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_x_test, W1) + b1), W2) + b2), W3) + b3)




In [18]:
num_steps = 1000001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (y_train.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = x_train[offset:(offset + batch_size), :]
    batch_labels = y_train[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_x_train : batch_data, tf_y_train : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
        print_stats(session, step, batch_data, batch_labels, loss, accuracy)
        #print(test_prediction.eval())
      #print("Minibatch loss at step %d: %f" % (step, l))
      #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      #print("Validation accuracy: %.1f%%" % accuracy(
       # valid_prediction.eval(), y_test))
        print("Test accuracy: %.1f%%" % give_accuracy(test_prediction.eval(), y_test))

Initialized
Loss at step 0 is: 1.081263
Batch accuracy at step 0 is: 0.822266
Test accuracy: 65.1%
Loss at step 1000 is: 0.107353
Batch accuracy at step 1000 is: 0.972656
Test accuracy: 73.2%
Loss at step 2000 is: 0.108900
Batch accuracy at step 2000 is: 0.962891
Test accuracy: 74.1%


KeyboardInterrupt: 